<img src="./images/kisti_logo.jpg" width="100"> <img src="./images/unist_logo.jpg" width="100"> <img src="./images/kisti_mas0.png" width="90"> <img src="./images/unist_mas.jpg" width="70">

# **2025년 겨울방학 AI 팀프로젝트 슈퍼컴퓨팅 청소년 캠프**
---
### **Edge Device를 이용한 AI 실습**
### **1. 훈련 이미지 수집**
---

이 노트북은 2025년 KISTI/UNIST 겨울방학 AI 팀프로젝트 슈퍼컴퓨팅 청소년 캠프에서, Edge Device를 활용한 AI 실습을 위한 데이터 수집, 훈련, 테스트 도구 입니다. 

NVIDIA Deep Learnning Institute course를 참고하여 작성되었으며, Jetson Orin Nano와 USB camera를 이용하여 실습이 가능하도록 작성되었습니다.

실습은 노트북의 셀을 순차적으로 적용하면서 진행됩니다.

---
### USB 카메라 연결 확인

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;">
    <p>USBCamera의 인스턴스는 한 번에 하나만 있을 수 있습니다. 이 셀을 시작하기 전에 실행되고 있는 카메라 인스턴스가 해제 되어있어야 합니다. 해제하기 위해 기존 인스턴스를 종료 하거나 커널을 종료해 주세요. </p>
</div>

In [ ]:
# 보드에 카메라 연결 확인
!ls -ltrh /dev/video*

In [ ]:
# jetcam 라이브러리를 이용한 USB 카메라 연결 설정
from jetcam.usb_camera import USBCamera

# usb 카메라 설정 (QVGA 사이즈, 가로:320, 세로:240 픽셀의 이미지를 video 0에서 획득)
camera = USBCamera(width=320, height=240, capture_device=0) 

camera.running = True
print("카메라 확인 됨")

---
### **(수정) 문제 설정**

문제를 설정을 위해 `TASK`와 수집할 데이터의 `CATEGORIES`를 정의. 선택한 이름으로 여러 데이터세트 `DATASETS`에 대한 공간을 정의.

기본 예제로 정의된 문제는 엄지를 든 손의 업/다운을 판단하는 분류(classification) 문제이다.

In [ ]:
# 문제 제목 설정
TASK = 'thumbs'

# 문제 수집 데이터 정의
CATEGORIES = ['thumbs_up', 'thumbs_down']

# 수집 데이터 세트 정의 (각 조별)
DATASETS = ['A', 'B']

# 데이터 저장 폴더 지정, 만약에 없는 경우 이후 생성
DATA_DIR = './classification/'

아래 셀 부터는 수정 금지

---
카테고리 정의 및 이미지 저장 폴더 확인

In [ ]:
from dataset import ImageClassificationDataset

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(DATA_DIR + TASK + '_' + name, CATEGORIES)
    
print("{} 문제를 위한 {} 카테고리 정의 완료".format(TASK, CATEGORIES))

# 데이터 저장 폴더 지정, 만약에 없는 경우 생성
!mkdir -p {DATA_DIR}

---
### Data 수집 위젯 생성

데이터 수집 도구 위젯을 생성하여 데이터 수집을 쉽게 할 수 있게 함.

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("데이터 수집 위젯 생성 완료")

---
### 실시간 데이터 수집

- 파일이 저장 될 위치에 대한 dataset을 설정
- 저장 할 category를 설정
- 카메라에서 생성되는 이미지를 설정한 category에 맞도록 조정 후, add 버튼을 통해 그림 저장
- 훈련 이미지 개수는 count에 나타나게 되며, 원하는 양에 맞게 계속 추가 

In [ ]:
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget]), 
    ])

display(all_widget)

---
### 카메라 및 프로그램 종료

- 카메라 인스턴스 종료 및 커널 종료

In [ ]:
# 카메라 종료 (리소스 해제), 커널 종료
import os
os._exit(00)